In [24]:
# Import CVX and numpy libraries
import cvxpy as cvx
import numpy as np
import random
import networkx as nx
import matplotlib.pyplot as plt

In [99]:
def generate(n,npoints):
    # Set the limits
    origA = np.full((n+1,n),-5.)
    rangA = np.full((n+1,n), 10.)
    origP = np.full((npoints,n),-5.)
    rangP = np.full((npoints,n), 10.)

    # Construct the anchor points
    a = np.random.rand(n+1,n)
    a = origA + np.multiply(a,rangA)

    # Construct the sensor points
    p = np.random.rand(npoints,n)
    p = origP + np.multiply(p,rangP)

    adjacency = np.full((npoints+n+1,npoints+n+1),-1.)

    if(n > 1):
        minConstr = 3
        maxConstr = 4
        minPointConstr = 2
        minAnchConstr = 2
    else:
        minConstr = 2
        maxConstr = 3
        minPointConstr = 1
        minAnchConstr = 1

    anchList = [i for i in range(n+1)]
    pointList = [i for i in range(npoints)]
    off = n + 1

    for i in range(npoints):
        random.shuffle(anchList)
        random.shuffle(pointList)
        # Set anchor constraints, between 1 and 3
        numAnch = random.randint(minAnchConstr,minConstr)
        for l in range(numAnch):
            adjacency[anchList[l],i+off] = np.linalg.norm(a[anchList[l]] - p[i])
            adjacency[i+off,anchList[l]] = adjacency[anchList[l],i+off]
        print(numAnch)
        # Count number of constraints already set for this point
        countConstr = 0
        for l in range(npoints+off):
            if(adjacency[l,i+off] > 0):
                countConstr = countConstr + 1
        # Set point constraints, between 0 and 3
        if(minConstr - countConstr >= 0):
            numPoint = random.randint(minConstr-countConstr,3)
            count = numPoint
            l = 0
            while(count > 0 and l < len(pointList)):
                if(pointList[l] != i):
                        countConstr = 0
                        for k in range(npoints+off):
                            if(adjacency[k,pointList[l]+off] > 0):
                                countConstr = countConstr + 1
                        if(countConstr < maxConstr):
                            adjacency[pointList[l]+off,i+off] = np.linalg.norm(p[pointList[l]] - p[i])
                            adjacency[i+off,pointList[l]+off] = adjacency[pointList[l]+off,i+off]
                            count = count - 1
                l = l + 1
    return [a, p, adjacency]

In [151]:
def generate_inside_hull(n,npoints):
    # Set the limits
    origA = np.full((n+1,n),-5.)
    rangA = np.full((n+1,n), 10.)
    origP = np.full((npoints,n),-5.)
    rangP = np.full((npoints,n), 10.)
    
    if(n == 1):
        a = np.array([-6, 6])
    elif(n == 2):
        a = np.array([[-6, 15], [-6, -15], [8, 0]])
    elif(n == 3):
        a = np.array([[-6, 15,-10], [-6, -15,-10], [8, 0, -10], [0, 0, 10]])
    # Construct the sensor points
    p = np.random.rand(npoints,n)
    p = origP + np.multiply(p,rangP)

    adjacency = np.full((npoints+n+1,npoints+n+1),-1.)

    if(n > 1):
        minConstr = 3
        maxConstr = 4
        minPointConstr = 2
        minAnchConstr = 2
    else:
        minConstr = 2
        maxConstr = 3
        minPointConstr = 1
        minAnchConstr = 1

    anchList = [i for i in range(n+1)]
    pointList = [i for i in range(npoints)]
    off = n + 1

    for i in range(npoints):
        random.shuffle(anchList)
        random.shuffle(pointList)
        # Set anchor constraints, between 1 and 3
        numAnch = random.randint(minAnchConstr,minConstr)
        for l in range(numAnch):
            adjacency[anchList[l],i+off] = np.linalg.norm(a[anchList[l]] - p[i])
            adjacency[i+off,anchList[l]] = adjacency[anchList[l],i+off]
        print(numAnch)
        # Count number of constraints already set for this point
        countConstr = 0
        for l in range(npoints+off):
            if(adjacency[l,i+off] > 0):
                countConstr = countConstr + 1
        # Set point constraints, between 0 and 3
        if(minConstr - countConstr >= 0):
            numPoint = random.randint(minConstr-countConstr,3)
            count = numPoint
            l = 0
            while(count > 0 and l < len(pointList)):
                if(pointList[l] != i):
                        countConstr = 0
                        for k in range(npoints+off):
                            if(adjacency[k,pointList[l]+off] > 0):
                                countConstr = countConstr + 1
                        if(countConstr < maxConstr):
                            adjacency[pointList[l]+off,i+off] = np.linalg.norm(p[pointList[l]] - p[i])
                            adjacency[i+off,pointList[l]+off] = adjacency[pointList[l]+off,i+off]
                            count = count - 1
                l = l + 1
    return [a, p, adjacency]

In [222]:
# p1 = np.array([-0.4, 0.2])
# p2 = np.array([-0.1, 1.7])
# p = [p1, p2]

# # We are working in R^2
# n = 2

# # Construct the anchor points
# a1 = np.array([1.0, 0.0])
# a2 = np.array([-1.0, 0.0])
# a3 = np.array([0.0, 2.0])
# a = [a1, a2, a3]

# #(a, p , matrix) = generate(n)

# d11 = np.linalg.norm(p1 - a1)
# d12 = np.linalg.norm(p1 - a2)
# d22 = np.linalg.norm(p2 - a2)
# d23 = np.linalg.norm(p2 - a3)
# dbar12 = np.linalg.norm(p1 - p2)
# matrix = [[0, 0, 0, d11, 0], [0, 0, 0, d12, d22], [0, 0, 0, 0, d23], [0, 0, 0, 0, dbar12], [0, 0, 0, 0, 0]]
n = 3
npoints = 10

(a, p, adjacency) = generate_inside_hull(n, npoints)
print a.shape
print a, p, d
# Compute the Euclidian distances to the anchor points
adjSize = len(p) + len(a)
asize = len(a)
d = []

for i in range(adjSize):
    for j in range(adjSize):
        if(j > i and adjacency[i][j] > 0 and i < asize):
            d.append((adjacency[i][j], j - asize, i, True))
        elif(j > i and adjacency[i][j] > 0):
            d.append((adjacency[i][j], i - asize, j - asize, False))

# Construct the CVX variables to minimize
x = [cvx.Variable(n) for i in range(len(p))]

print d

states = []
constr = []

cost = cvx.norm(0)
for (distance, i, j, truth) in d:
    if(truth):
        constr.append(cvx.norm(x[i] - a[j], 2) ** 2 <= distance ** 2)
    else:
        constr.append(cvx.norm(x[i] - x[j], 2) ** 2 <= distance ** 2)
        
states.append(cvx.Problem(cvx.Minimize(cost), constr))

# Add the states and solve the problem    
prob = sum(states)
prob.solve()

if(n == 1): #Solves it more or less
    print('Solution: ' + prob.status)
    for i, elem in enumerate(x):
        soln1 = elem.value
        point1 = p[i]
        print("Sensor " + str(i) + " is at " + str(soln1) + " and the actual value is " + str(point1))
elif(n == 2): #Can't solve this
    print('Solution: ' + prob.status)
    for i, elem in enumerate(x):
        soln1 = elem.value.A.flatten()
        soln1 = ['%.4f' % elem for elem in soln1]
        soln1 = [float(elem) for elem in soln1]
        point1 = ['%.4f' % elem for elem in p[i]]
        point1 = [float(elem) for elem in point1]
        print("Sensor " + str(i) + " is at " + str(soln1) + " and the actual value is " + str(point1))
else:
    print('Solution: ' + prob.status)
    for i, elem in enumerate(x):
        soln1 = elem.value.A.flatten()
        soln1 = ['%.4f' % elem for elem in soln1]
        soln1 = [float(elem) for elem in soln1]
        point1 = ['%.4f' % elem for elem in p[i]]
        point1 = [float(elem) for elem in point1]
        print("Sensor " + str(i) + " is at " + str(soln1) + " and the actual value is " + str(point1))

3
3
2
3
3
3
2
2
2
3
(4, 3)
[[ -6  15 -10]
 [ -6 -15 -10]
 [  8   0 -10]
 [  0   0  10]] [[ 0.90204417 -0.5608415  -2.10131799]
 [-1.12469802 -3.33703014 -2.34741597]
 [ 0.30753296 -3.9534824   2.96435901]
 [-1.27636942  0.26487563 -0.19049378]
 [ 0.81090697  0.0696528   2.77530177]
 [ 2.70430178 -4.86757561  1.62406987]
 [ 1.0901596  -1.26908754 -4.74832192]
 [ 2.12089472  2.08494649 -2.12426946]
 [ 2.16357779  0.40242724  2.06258759]
 [ 0.72009882  1.2466785   2.90480377]] [(23.625878094713986, 0, 0, True), (15.058500435358541, 1, 0, True), (22.8152378603512, 4, 0, True), (17.601959350359468, 6, 0, True), (18.821762007702635, 8, 0, True), (18.538019455340894, 1, 1, True), (17.024489262586478, 2, 1, True), (18.937315627683358, 7, 1, True), (23.382601369198188, 8, 1, True), (24.70117419302829, 9, 1, True), (14.47950556955572, 0, 2, True), (13.628881773172679, 1, 2, True), (13.026778435582772, 2, 2, True), (15.305074706459687, 3, 2, True), (13.526999020125954, 4, 2, True), (15.3904612988

In [223]:
%pylab inline --no-import-all
if(n == 2):
    G=nx.Graph()

    for i in range(n+1):
        G.add_node(i,pos=a[i])

    for i in range(npoints):
        G.add_node(n+1+i,pos=p[i])

    pos=nx.get_node_attributes(G,'pos')

    nx.draw_networkx_nodes(G,pos,
                           nodelist=[i for i in range(n+1)],
                           node_color='r',
                           node_size=50,
                           alpha=0.8)
    nx.draw_networkx_nodes(G,pos,
                           nodelist=[i for i in range(n+1,npoints+n+1)],
                           node_color='b',
                           node_size=50,
                           alpha=0.8)
    for i in range(npoints+n+1):
        for j in range(n+1,npoints+n+1):
            if(adjacency[j,i] > 0.):
                G.add_edge(i,j)

    nx.draw_networkx_edges(G,pos)
    plt.draw()  # pyplot draw()

Populating the interactive namespace from numpy and matplotlib


In [224]:
T = n + npoints

# We are working in R^2
            
#This creates a 3x3 semipositive definite matrix which we will 
#use as part of our constraints
z = cvx.Semidef(T)

eyeConstraint = []
anchorConstraints = []
pointConstraints = []

for i in range(n):
    temp = np.zeros((T,T))
    temp[i][i] = 1
    eyeConstraint.append(temp)
    
temp = np.zeros((T,T))
for i in range(n):
    for j in range(n):
        temp[i][j] = 1
eyeConstraint.append(temp)

for (distance, i, j, truth) in d:
    if truth:
        temp = np.zeros(npoints)
        temp[i] = -1.
        anchorConstraints.append((np.outer(np.append(a[j], temp), np.append(a[j], temp)), distance))
    else:
        tempi = np.zeros(npoints)
        tempj = np.zeros(npoints)
        tempi[i] = 1.
        tempj[j] = 1.
        temp = tempi - tempj
        corner = np.zeros(n)
        temp = np.append(corner, temp)
        pointConstraints.append((np.outer(temp,temp), distance))

print d

#Another empty states list
states = []

cost = cvx.norm(0)

#The four constraints in the SDP relaxation problem
#Note that the last constraint forces z to be SPD
constr = []

for i, mat in enumerate(eyeConstraint):
    if i < len(eyeConstraint) - 1:
        constr.append(cvx.sum_entries(cvx.mul_elemwise(mat, z)) == 1)
    else:
        constr.append(cvx.sum_entries(cvx.mul_elemwise(mat, z)) == n)

for mat in anchorConstraints:
    constr.append(cvx.sum_entries(cvx.mul_elemwise(mat[0], z)) ==  mat[1] ** 2)

for mat in pointConstraints:
    constr.append(cvx.sum_entries(cvx.mul_elemwise(mat[0], z)) ==  mat[1] ** 2)

constr.append(z >> 0)

#Add the constraints and cost function
states.append(cvx.Problem(cvx.Minimize(cost), constr))

#Solve the SDP relaxation problem
prob = sum(states)
prob.solve();    

print('Solution: ' + prob.status)

for i in range(npoints):
    soln1 = z.value.A[0:n, i + n]
    point1 = p[i]
    print("Sensor " + str(i) + " is located at " + str(soln1) + " and the actual value is " + str(point1))

[(18.766117854600786, 0, 0, True), (18.321107730830001, 3, 0, True), (24.609031481216903, 5, 0, True), (20.021177305412984, 9, 0, True), (17.847035873190841, 0, 1, True), (14.776856152348383, 1, 1, True), (18.162740095212239, 2, 1, True), (20.897158288410573, 4, 1, True), (17.707340113482505, 5, 1, True), (20.490765214963776, 7, 1, True), (21.19869775905709, 8, 1, True), (21.809361540054692, 9, 1, True), (10.634128911316516, 0, 2, True), (12.367616035155788, 1, 2, True), (15.584565340734972, 2, 2, True), (13.503614370254803, 3, 2, True), (14.659339873672119, 4, 2, True), (13.669554208949998, 5, 2, True), (8.7713511050283941, 6, 2, True), (12.839758446007595, 1, 3, True), (10.27353110733945, 3, 3, True), (7.270397932687569, 4, 3, True), (14.842911796279632, 6, 3, True), (12.483713638627114, 7, 3, True), (8.236839949098119, 8, 3, True), (7.2397899936341101, 9, 3, True), (3.0130621795650301, 0, 3, False), (6.4493637179409786, 1, 4, False), (5.7250771806905059, 1, 5, False), (6.65164872137